# Initialization

In [1]:
import pandas as pd
import numpy as np
import functools
import collections
import operator
pd.options.mode.chained_assignment = None

df = pd.read_excel('song_data2.xlsx')

# Replace NaN values with empty strings

df['entity'] = df['entity'].fillna('')
df['Google_Entities'] = df['Google_Entities'].fillna('')

# Function to convert the genre value in each row to a list  
def list_converter (value):
    return value.split(', ') 
df['genre'] = df['genre'].apply(list_converter)


# Here we define the user interface function the includes all the filters created above 




## Define User Interface

In [2]:
def UserInterface1(df):
    
    print("In the following steps you will filter the song dataset on the following variables: ")
    [print(i) for i in df.columns.values]
    print("")
    
    ####################
    ###    ARTIST    ###
    ####################
    
    unique_artists = list(df['artist'].unique())
    print ('\nArtist options: ', unique_artists)
    artist_input = input('\nPlease select your artists (or type * to select all): ')

    if artist_input == '*':
        artist_input = unique_artists
    else:
        artist_input = artist_input.split(', ')
    
    # Filtering for artist
    df_filtered = df.loc[df['artist'].isin(artist_input)]
    
    ####################
    ###     ALBUM    ###
    ####################
    
    unique_albums = list(df_filtered['album'].unique())
    print ('\nAlbum options: ', unique_albums)
    album_input = input('\nPlease select your albums (or type * to select all): ')
    
    if album_input == '*':
        album_input = unique_albums
    else:
        album_input = album_input.split(', ')
    
    df_filtered = df_filtered.loc[df_filtered['album'].isin(album_input)]
    
    ####################
    ###   COUNTRY    ###
    ####################
    
    unique_countries = list(df_filtered['home_country'].unique())
    print ('\nCountry options: ', unique_countries)
    country_input = input('\nPlease select your countries (or type * to select all): ')
    
    if country_input == '*':
        country_input = unique_countries
    else:
        country_input = country_input.split(', ')
    
    df_filtered = df_filtered.loc[df_filtered['home_country'].isin(country_input)]
    
    ###################
    ###    GENRE    ###
    ###################

    unflattened_genre_list = list(df_filtered['genre'])
    unique_genres = set([item for sublist in unflattened_genre_list for item in sublist])
    print ('\nGenre options: ', unique_genres)
    
    genre_input = input('\nPlease select your genres (or type * to select all): ')
    
    if genre_input == '*':
        genre_input = list(unique_genres)
    else:
        genre_input = genre_input.split(', ')
    
    df_filtered['genre_match'] = False
    for count, each_row in enumerate(df_filtered['genre']):
        for item in each_row:
            if item in genre_input:
                df_filtered['genre_match'].iloc[count] = True
    df_filtered = df_filtered.loc[df_filtered['genre_match'] == True]
    
    
    ####################
    ### RELEASE YEAR ###
    ####################
    
    print ('\nRelease date options: \n 1960-1969 \n 1970-1979 \n 1980-1989 \n 1990-1999 \n 2000-2009 \n 2010-Present')
    
    release_input = input('\nPlease select your release year (or type * to select all): ')
    
    release_year_list = []

    if release_input == '*':
        for elem in range (1960, 2018):
            release_year_list.append(elem)
    else:
        release_input = release_input.split(', ')
        for elem in release_input:
            if '1960' in elem:
                    for elem in range (1960, 1970):
                        release_year_list.append(elem)
            elif '1970' in elem:
                    for elem in range (1970, 1980):
                        release_year_list.append(elem)
            elif '1980' in elem:
                    for elem in range (1980, 1990):
                        release_year_list.append(elem)
            elif '1990' in elem:
                    for elem in range (1990, 2000):
                        release_year_list.append(elem)
            elif '2000' in elem:
                    for elem in range (2000, 2010):
                        release_year_list.append(elem)
            elif '2010' in elem:
                    for elem in range (2010, 2018):
                        release_year_list.append(elem)

    release_input = release_year_list
    
    df_filtered = df_filtered.loc[df_filtered['released'].isin(release_input)]
    
    ##################
    ### SINGER AGE ###
    ##################
    
    print ('\nSinger age options: \n 10-19 \n 20-29 \n 30-39 \n 40-49 \n 50-59')
    
    age_input = input('\nPlease select an age range for singers (or type * to select all): ')
    
    age_year_list = []

    if age_input == '*':
        for elem in range (10, 56):
            age_year_list.append(elem)
    else:
        age_input = age_input.split(', ')
        for elem in age_input:
            if '10' in elem:
                    for elem in range (10, 20):
                        age_year_list.append(elem)
            elif '20' in elem:
                    for elem in range (20, 30):
                        age_year_list.append(elem)
            elif '30' in elem:
                    for elem in range (30, 40):
                        age_year_list.append(elem)
            elif '40' in elem:
                    for elem in range (40, 50):
                        age_year_list.append(elem)
            elif '50' in elem:
                    for elem in range (50, 57):
                        age_year_list.append(elem)

    age_input = age_year_list
    
    df_filtered = df_filtered.loc[df_filtered['singer_age_at_release'].isin(age_input)]
    
    
    #     ____        _               _   
    #    / __ \      | |             | |  
    #   | |  | |_   _| |_ _ __  _   _| |_ 
    #   | |  | | | | | __| '_ \| | | | __|
    #   | |__| | |_| | |_| |_) | |_| | |_ 
    #    \____/ \__,_|\__| .__/ \__,_|\__|
    #                    | |              
    #                    |_|              
    
    
    print("\n===================================================================\n===================================================================\n")
    
    ## Print average sentiment of songs reslting from filtering ##
    ####
    
    print ('The average sentiment of the songs resulting from your search is: ' + str(round(df_filtered['overall_sentiment'].mean(), 2)))
    
    print("\n===================================================================\n")
    
    ## Return the Top 5 topics and their sentiment for the filtered songs
    ####
    
    print('\nThe top 5 most prevalent topics of the songs resulting from your search are:\n')
    
    df_sentiment_copy = df_filtered.copy()
    df_sentiment_copy = df_sentiment_copy.sort_values(by=['overall_sentiment'], axis=0, ascending=False)[:5]
    df_sentiment_copy = df_sentiment_copy.reset_index()
    
    TopTopics = []
    for i in df['Google_Entities']:
        if i != '':
            for L in eval(i):
                for key,value in L.items():
                    TopTopics.append([key,value])

    li = sorted(TopTopics, key=operator.itemgetter(1), reverse = True)   # Ascending order

    for i in li[:5]:
        print(i[0])
    
    
    #counter = 0
    #while counter < len(df_sentiment_copy['artist']):
    #    print ('Song: "' + (df_sentiment_copy['song'][counter]) + '", Artist: ' + str((df_sentiment_copy['artist'][counter])) + ', '  
    #    + 'Sentiment: ' + str((df_sentiment_copy['overall_sentiment'][counter])))  
    #    print("")
    #    counter +=1
    #
    return df_filtered




## Run User Interface 1

In [3]:
# Test User Interface
df_filtered = UserInterface1(df)

In the following steps you will filter the song dataset on the following variables: 
ID
artist
song
album
released
genre
home_country
singer_age_at_release
entity
topic
Google_Entities
overall_sentiment


Artist options:  ['Aerosmith', 'Bruno Mars', 'Coldplay', 'Doors', 'Elton John', 'Elvis Presley', 'Grateful Dead', 'Jimi Hendrix', 'John Legend', 'Lady Gaga', 'Linkin Park', 'Maroon 5', 'Metallica', 'Michael Jackson', 'Nickelback', 'Outkast', 'Santana', 'Stevie Wonder', 'Weezer', 'Wu-Tang Clan']

Please select your artists (or type * to select all): Aerosmith

Album options:  ['Pump', 'Permanent Vacation', 'Nine Lives', 'Aerosmith', "Honkin' on Bobo", 'Just Push Play', 'Draw the Line', 'Rock in a Hard Place']

Please select your albums (or type * to select all): *

Country options:  ['United States']

Please select your countries (or type * to select all): *

Genre options:  {'Hard rock', 'Rock', 'Classic rock'}

Please select your genres (or type * to select all): *

Release date opti

In [8]:
# df_filtered.head()

## Define User Interface 2

In [3]:
def makeInvertedIndex(df,voctrain):
       
    InvertInd = {}
    
    for word in voctrain:
        InvertInd[word] = []
    

    for j in voctrain:
        for ind, val in enumerate(df['topic']):
            if df['topic'][ind] != '':
                evalue = eval(val) 
                #print(evalue)
                for dual in evalue:
                    for key,value in dual.items():
                        keySplit = key.split(' ')
                        for kS in keySplit:
                            if j == kS:
                                InvertInd[j].append(ind)
    
        
    for j in voctrain:
        for ind, val in enumerate(df['Google_Entities']):
            if df['Google_Entities'][ind] != '':
                evalue = eval(val) 
                for dual in evalue:
                    for key,value in dual.items():
                        keySplit = key.split(' ')
                        for kS in keySplit:
                            if j == kS:
                                InvertInd[j].append(ind)  
    

    for j in voctrain:
        for ind, val in enumerate(df['entity']):
            if df['entity'][ind] != '':
                evalue = eval(val) 
                for dual in evalue:
                    for key,value in dual.items():
                        keySplit = key.split(' ')
                        for kS in keySplit:
                            if j == kS:
                                InvertInd[j].append(ind)
             
        
        
    return InvertInd          



def orSearch(invertedIndex, query, df):
    
    results = []
    
    print("Query is:",query,"\n")
    
    for ask in query:
        print("Check for:",ask)
        if ask in invertedIndex:
            word = invertedIndex[ask]
            print("Found...\n")
            for res in word:
                #print(res)
                if res not in results:
                    results.append(res)
        else:
            print("Not found...\n")
        
    for i in results:
        print(df['song'].iloc[i]," by ", df['artist'].iloc[i])
    #print("Results: ",results)

def andSearch(invertedIndex, query, df):
    
    results = {}
    resultCheck = []
    
    for ask in query:
        results[ask] = []
    
    print("Query is:",query,"\n")
    
    Match = False
    
    for ask in query:
        print("Check for:",ask,"...")
        if ask in invertedIndex:
            word = invertedIndex[ask]
            print("Found\n")
            for res in word:
                results[ask].append(res)
                if res not in resultCheck:
                    resultCheck.append(res)
        else:
            print("Not found\n")
    
    print("Results: ",results)
    
    Matches = []
    
    for check in resultCheck:
        Check = True
        for key,value in results.items():
            if check in value:
                Check = Check and True
            else:
                Check = Check and False
        if Check == True:
            Matches.append(check)
    
    Matches = set (Matches)
    print('\n===================================\n')
    print("Common matches:",Matches,"\n")
    
    for num in Matches:
        print("Found in title: ",df.iloc[num][2], " by ", df.iloc[num][1])
    
def VocBuilder(df):    
    # Create vocabulary for 'entity'

    entity_vocabulary = []

    for i in range(len(df['entity'])):

        if df['entity'][i] != '':
            test = eval(df['entity'][i])
            for j in test:
                for key,value in j.items():
                    temp = key.split(' ')
                    for item in temp:
                        entity_vocabulary.append(item)

    entity_vocabulary = list(set(entity_vocabulary))
    #print(list(set(entity_vocabulary)))


    # Create vocabulary for 'topic'

    topic_vocabulary = []

    for i in range(len(df['topic'])):
        test = eval(df['topic'][i])
        for j in test:
            for key,value in j.items():
                temp = key.split(' ')
                for item in temp:
                    topic_vocabulary.append(item)

    topic_vocabulary = list(set(topic_vocabulary))
    #print(list(set(topic_vocabulary)))


    # Create vocabulary for 'Google_Entities'

    google_entity_vocabulary = []

    for i in range(len(df['topic'])):
        test = eval(df['topic'][i])
        for j in test:
            for key,value in j.items():
                temp = key.split(' ')
                for item in temp:
                    google_entity_vocabulary.append(item)

    google_entity_vocabulary = list(set(google_entity_vocabulary))
    
    # Create vocabularies of individual columns 'entity','topic' and 'Google_Entity' 
    vocabulary_train = entity_vocabulary + topic_vocabulary + google_entity_vocabulary
    vocabulary_train = list(set(vocabulary_train))    
    
    return vocabulary_train
    
def UserInterface2(df):
        
   
    print("In the following steps you will be able to search songs, artists, and albums by Topics and Entities: ")
    print("")
    print ("Lookup topics such as Mars, Fire, Weapons, Love, Death")
    topics_input = input('\nPlease enter topics  (or type * to select some default topics): ')

    if topics_input == '*':
        topics_input = ['Love', 'War']
    else:
        topics_input = topics_input.split(', ')
    
    print ("Would you like to filter the song dataset on the following variables")
    filter_input = input('\nPlease select 1 to filter or 0 to search entire dataset: ')
    if filter_input == '1':
        df_filtered = UserInterface1(df)
        choice_df = df_filtered.reset_index()
        #print(choice_df.head())
    else:
        choice_df = df
        
    vocabulary_train = VocBuilder(choice_df)
    
    # take the choice of df, either the filtered one or the entire df and pass to inverted index.    
    invertind = makeInvertedIndex(choice_df,vocabulary_train)
    
    andor_input = input('\nPlease select OR to perform OR search, AND to perform AND search!: ')
    # search inverted index using whichever method AND/OR
    if andor_input == 'OR':
        orSearch(invertind,topics_input, choice_df)
    else:
        andSearch(invertind,topics_input, choice_df)
    

# Run User Interface 2

In [19]:
UserInterface2(df)

In the following steps you will be able to search songs, artists, and albums by Topics and Entities: 

Lookup topics such as Mars, Fire, Weapons, Love, Death

Please enter topics  (or type * to select some default topics): God
Would you like to filter the song dataset on the following variables

Please select 1 to filter or 0 to search entire dataset: 1
In the following steps you will filter the song dataset on the following variables: 
ID
artist
song
album
released
genre
home_country
singer_age_at_release
entity
topic
Google_Entities
overall_sentiment


Artist options:  ['Aerosmith', 'Bruno Mars', 'Coldplay', 'Doors', 'Elton John', 'Elvis Presley', 'Grateful Dead', 'Jimi Hendrix', 'John Legend', 'Lady Gaga', 'Linkin Park', 'Maroon 5', 'Metallica', 'Michael Jackson', 'Nickelback', 'Outkast', 'Santana', 'Stevie Wonder', 'Weezer', 'Wu-Tang Clan']

Please select your artists (or type * to select all): Aerosmith

Album options:  ['Pump', 'Permanent Vacation', 'Nine Lives', 'Aerosmith', "Ho

# Define Main User Interface

In [20]:
def UserInterface3(df):
    print ("WELCOME!\n")
    key_choice = input('\nTo Perform Filtered Search press 1, To perform Topic Search Press 2: ')

    if key_choice == '1':
        df_filtered = UserInterface1(df)
    else:
        UserInterface2(df)
#     else: 
#         print("Did not catch that!\n")
#         key_choice = input('\nTo Perform Filtered Search press 1, To perform Topic Search Press 2: ')

# Run Main User Interface

In [22]:
UserInterface3(df)

WELCOME!


To Perform Filtered Search press 1, To perform Topic Search Press 2: 1
In the following steps you will filter the song dataset on the following variables: 
ID
artist
song
album
released
genre
home_country
singer_age_at_release
entity
topic
Google_Entities
overall_sentiment


Artist options:  ['Aerosmith', 'Bruno Mars', 'Coldplay', 'Doors', 'Elton John', 'Elvis Presley', 'Grateful Dead', 'Jimi Hendrix', 'John Legend', 'Lady Gaga', 'Linkin Park', 'Maroon 5', 'Metallica', 'Michael Jackson', 'Nickelback', 'Outkast', 'Santana', 'Stevie Wonder', 'Weezer', 'Wu-Tang Clan']

Please select your artists (or type * to select all): Metallica, Nickelback

Album options:  ['St. Anger', 'All the Right Reasons', 'Curb', 'No Fixed Address']

Please select your albums (or type * to select all): *

Country options:  ['United States']

Please select your countries (or type * to select all): *

Genre options: 

Please select your genres (or type * to select all): *

Release date options: 
 1960-1

In [6]:
#df